<a href="https://colab.research.google.com/github/paulgureghian/Google_Colab_PyTorch_Notebooks/blob/master/Medetec_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
from torch import nn, optim
from collections import OrderedDict
from torch.optim import lr_scheduler 
from torchvision import  models, datasets, transforms

In [0]:
data_dir = '/content/drive/My Drive/Medetec/data/medetec_dataset'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [0]:
training_transforms= transforms.Compose([transforms.RandomRotation(45),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.RandomCrop(224),
                                         transforms.RandomResizedCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485,0.456,0.406),
                                                                 (0.229,0.224,0.255))])

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=32) 

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = models.vgg19(pretrained=True)


for param in model.parameters():
    param.requires_grad_(False) 
    
model.classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(25088, 4096)),
                                ('relu1', nn.ReLU()),                                                             
                                ('fc2', nn.Linear(4096, 15)), 
                                ('output', nn.Softmax(dim=1))
                                ])) 

model.to(device) 
epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) 

In [8]:
for epoch in range(epochs):
    scheduler.step()
    train_loss = 0
    
    model.train()
    for inputs, labels in training_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        
        loss = criterion(logps, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
    model.eval()
    accuracy = 0
    valid_loss = 0
    
    for inputs, labels in validation_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        
        loss = criterion(output, labels)
        
        valid_loss += loss.item()
        
        ps = torch.exp(output)
        top_ps, top_class = ps.topk(1, dim=1) 
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    print('Epoch {}/{}.. '.format(epoch+1, epochs),
          'Training loss: {:.3f}.. '.format(train_loss / len(training_dataloader)),
          'Validation loss: {:.3f}.. '.format(valid_loss / len(validation_dataloader)),
          'Validation accuracy: {:.3f}.. '.format(accuracy / len(validation_dataloader)))
          
    model.train()      

Epoch 1/50..  Training loss: 2.568..  Validation loss: 2.520..  Validation accuracy: 0.300.. 
Epoch 2/50..  Training loss: 2.516..  Validation loss: 2.495..  Validation accuracy: 0.323.. 
Epoch 3/50..  Training loss: 2.494..  Validation loss: 2.489..  Validation accuracy: 0.316.. 
Epoch 4/50..  Training loss: 2.495..  Validation loss: 2.512..  Validation accuracy: 0.300.. 
Epoch 5/50..  Training loss: 2.488..  Validation loss: 2.516..  Validation accuracy: 0.300.. 
Epoch 6/50..  Training loss: 2.524..  Validation loss: 2.524..  Validation accuracy: 0.292.. 
Epoch 7/50..  Training loss: 2.537..  Validation loss: 2.537..  Validation accuracy: 0.269.. 
Epoch 8/50..  Training loss: 2.493..  Validation loss: 2.507..  Validation accuracy: 0.308.. 
Epoch 9/50..  Training loss: 2.504..  Validation loss: 2.535..  Validation accuracy: 0.276.. 
Epoch 10/50..  Training loss: 2.482..  Validation loss: 2.511..  Validation accuracy: 0.316.. 
Epoch 11/50..  Training loss: 2.509..  Validation loss: 2.5